# Importing

In [37]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score,RandomizedSearchCV,GridSearchCV
from scipy.stats import randint

## Importing trainning data

In [15]:
X_train=pd.read_csv("D:/python/ML/ML/Machine Learning Project/Telco customer churn/dataset/X_train_SMOTE.csv")
y_train=pd.read_csv("D:/python/ML/ML/Machine Learning Project/Telco customer churn/dataset/y_train_SMOTE.csv").values.ravel()

In [16]:
X_train

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges
0,1,0,1,1,0.880735,1,2,0,0,2,0,2,2,0,1,1,2,0.197365,0.657480
1,1,0,0,0,-1.277445,1,0,1,0,0,0,0,2,0,0,1,2,0.524739,-0.970243
2,1,0,0,0,-0.788800,1,0,2,1,1,1,1,1,1,1,0,1,-1.510962,-0.891227
3,0,1,1,1,-0.340876,1,0,1,0,0,2,0,2,2,2,1,2,1.056514,-0.007184
4,0,1,0,0,-1.073843,1,0,1,2,0,0,0,0,0,0,1,2,0.310367,-0.806850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7265,0,0,0,0,-0.803035,0,1,0,0,0,0,0,0,0,0,1,2,-1.356862,-0.876499
7266,0,0,0,0,-0.964211,1,2,1,0,2,0,0,0,0,0,0,0,0.537096,-0.687093
7267,0,0,0,0,-0.619378,0,1,0,2,0,0,0,0,2,0,0,0,-0.207575,-0.569464
7268,1,0,0,0,-1.277445,1,0,1,0,0,0,0,0,0,0,1,2,0.181673,-0.974797


In [17]:
X_train.shape,y_train.shape

((7270, 19), (7270,))

## training with deafult hyperparmeter

In [8]:
models = {
    "random_forest": RandomForestClassifier(random_state=42),
    "xgboost": XGBClassifier(random_state=42),
    "logistic_regression": LogisticRegression(random_state=42),
    "svm": SVC(random_state=42)
}

In [18]:
# it will store cross validation results
cv_scores={}

# perform 5-fold cross validation for each model
for model_name,model in models.items():
    print(f"Model is training : {model_name}")
    scores=cross_val_score(model,X_train,y_train,cv=5,scoring="accuracy")
    cv_scores[model_name]=scores

cv_scores

Model is training : random_forest
Model is training : xgboost
Model is training : logistic_regression
Model is training : svm


{'random_forest': array([0.79298487, 0.82943604, 0.8610729 , 0.86519945, 0.86038514]),
 'xgboost': array([0.75103164, 0.78060523, 0.85350757, 0.85213205, 0.86313618]),
 'logistic_regression': array([0.73452545, 0.75309491, 0.79711142, 0.78885832, 0.79298487]),
 'svm': array([0.76066025, 0.76616231, 0.81568088, 0.80467675, 0.81774415])}

### As  random_forest and xgboost is performing well lets try hyperparameter for this 2 models

In [20]:
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}
grid=GridSearchCV(RandomForestClassifier(),param_grid=param_dist,cv=5,scoring="roc_auc")
grid.fit(X_train,y_train)


,estimator,RandomForestClassifier()
,param_grid,"{'max_depth': [5, 10, ...], 'max_features': ['sqrt', 'log2'], 'min_samples_leaf': [1, 2, ...], 'min_samples_split': [2, 5, ...], ...}"
,scoring,'roc_auc'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,n_estimators,300


In [25]:
grid.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

In [26]:
grid.best_score_

np.float64(0.9176024399796416)

In [28]:
param_dist = {
    'n_estimators': [100, 300, 500],
    'max_depth': [5, 10, 15, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2']
}
random_search_random=RandomizedSearchCV(RandomForestClassifier(),param_distributions=param_dist,cv=5,scoring="roc_auc")
random_search_random.fit(X_train,y_train)
print(random_search_random.best_params_," values ->  ", random_search_random.best_score_)

{'n_estimators': 300, 'min_samples_split': 2, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'max_depth': 15}  values ->   0.9083760777554307


In [23]:
param_dist = {
    'n_estimators': [200, 400, 600],
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 5, 7],
    'min_child_weight': [1, 3, 5],
    'subsample': [0.8, 0.9, 1.0],
    'colsample_bytree': [0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.3],
}
random_search_XGboost = RandomizedSearchCV(XGBClassifier(n_jobs=-1),param_distributions=param_dist,n_iter=50,cv=5,scoring="roc_auc",random_state=42)
random_search_XGboost.fit(X_train,y_train)



,estimator,"XGBClassifier...ree=None, ...)"
,param_distributions,"{'colsample_bytree': [0.8, 0.9, ...], 'gamma': [0, 0.1, ...], 'learning_rate': [0.1, 0.05, ...], 'max_depth': [3, 5, ...], ...}"
,n_iter,50
,scoring,'roc_auc'
,n_jobs,None
,refit,True
,cv,5
,verbose,0
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


In [24]:
print(random_search_XGboost.best_params_, random_search_XGboost.best_score_)

{'subsample': 0.9, 'n_estimators': 400, 'min_child_weight': 1, 'max_depth': 7, 'learning_rate': 0.1, 'gamma': 0.3, 'colsample_bytree': 0.9} 0.9099460956730852


In [29]:
param_grid = {
    'n_estimators': [300, 400, 500],
    'max_depth': [5, 7, 9],
    'learning_rate': [0.05, 0.1],
    'min_child_weight': [1, 3],
    'subsample': [0.8, 0.9],
    'colsample_bytree': [0.8, 0.9],
    'gamma': [0.1, 0.3, 0.5]
}

Xggrid = GridSearchCV(
    XGBClassifier(use_label_encoder=False, eval_metric='logloss'),
    param_grid,
    cv=5,
    scoring='roc_auc',
    n_jobs=-1
)
Xggrid.fit(X_train, y_train)
print(Xggrid.best_params_, Xggrid.best_score_)


{'colsample_bytree': 0.8, 'gamma': 0.1, 'learning_rate': 0.05, 'max_depth': 9, 'min_child_weight': 1, 'n_estimators': 500, 'subsample': 0.9} 0.9143984530650163


In [30]:
grid.best_score_,Xggrid.best_score_

(np.float64(0.9176024399796416), np.float64(0.9143984530650163))

In [31]:
random_search_random.best_score_,random_search_XGboost.best_score_

(np.float64(0.9083760777554307), np.float64(0.9099460956730852))

## Random forest work slightly good
1. i will test both model test accuracy 
2. make a stack model

In [32]:
X_test=pd.read_csv("D:/python/ML/ML/Machine Learning Project/Telco customer churn/dataset/X_test.csv")
y_test=pd.read_csv("D:/python/ML/ML/Machine Learning Project/Telco customer churn/dataset/y_test.csv")

In [33]:
X_test.shape,y_test.shape

((2113, 19), (2113, 1))

In [40]:
y_test.value_counts()

Churn
0        1539
1         574
Name: count, dtype: int64

In [34]:
random_forest_model=grid.best_estimator_
random_forest_test_predict=random_forest_model.predict(X_test)

In [35]:
Xgboost_model=Xggrid.best_estimator_
Xgboost_test_predict=Xgboost_model.predict(X_test)

## Randomforest metrics

In [38]:
print(f"Accuracy : {accuracy_score(random_forest_test_predict,y_test)}")
print(f"confusion_matrix : {confusion_matrix(random_forest_test_predict,y_test)}")
print(f"classification_report : {classification_report(random_forest_test_predict,y_test)}")

Accuracy : 0.7605300520586843
confusion_matrix : [[1246  213]
 [ 293  361]]
classification_report :               precision    recall  f1-score   support

           0       0.81      0.85      0.83      1459
           1       0.63      0.55      0.59       654

    accuracy                           0.76      2113
   macro avg       0.72      0.70      0.71      2113
weighted avg       0.75      0.76      0.76      2113



## XGboost metrics

In [39]:
print(f"Accuracy : {accuracy_score(Xgboost_test_predict,y_test)}")
print(f"confusion_matrix : {confusion_matrix(Xgboost_test_predict,y_test)}")
print(f"classification_report : {classification_report(Xgboost_test_predict,y_test)}")

Accuracy : 0.7557974443918599
confusion_matrix : [[1237  214]
 [ 302  360]]
classification_report :               precision    recall  f1-score   support

           0       0.80      0.85      0.83      1451
           1       0.63      0.54      0.58       662

    accuracy                           0.76      2113
   macro avg       0.72      0.70      0.70      2113
weighted avg       0.75      0.76      0.75      2113



## The Model is overfitting

### trying to change some parameter

In [41]:
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_depth=15,
    min_samples_split=2,
    min_samples_leaf=2,
    max_features='sqrt',
    class_weight='balanced',   # fixes class imbalance
    n_jobs=-1,
    random_state=42
)

rf_model.fit(X_train, y_train)

,n_estimators,300
,criterion,'gini'
,max_depth,15
,min_samples_split,2
,min_samples_leaf,2
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [42]:
xgb_model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.05,
    max_depth=9,
    min_child_weight=1,
    subsample=0.9,
    colsample_bytree=0.8,
    gamma=0.1,
    scale_pos_weight=2.68,     # ~1539 / 574 → balances churn
    use_label_encoder=False,
    eval_metric='logloss',
    n_jobs=-1,
    random_state=42
)

xgb_model.fit(X_train, y_train)

,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,0.8
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [44]:
from sklearn.metrics import roc_auc_score
for name, model in {"RandomForest": rf_model, "XGBoost": xgb_model}.items():
    print(f"\n{name}")
    preds = model.predict(X_test)
    probs = model.predict_proba(X_test)[:, 1]
    
    print("Confusion Matrix:\n", confusion_matrix(y_test, preds))
    print("AUC:", roc_auc_score(y_test, probs))
    print(classification_report(y_test, preds, digits=3))



RandomForest
Confusion Matrix:
 [[1216  323]
 [ 171  403]]
AUC: 0.8375902493360774
              precision    recall  f1-score   support

           0      0.877     0.790     0.831      1539
           1      0.555     0.702     0.620       574

    accuracy                          0.766      2113
   macro avg      0.716     0.746     0.726      2113
weighted avg      0.789     0.766     0.774      2113


XGBoost
Confusion Matrix:
 [[1165  374]
 [ 169  405]]
AUC: 0.8226918923324571
              precision    recall  f1-score   support

           0      0.873     0.757     0.811      1539
           1      0.520     0.706     0.599       574

    accuracy                          0.743      2113
   macro avg      0.697     0.731     0.705      2113
weighted avg      0.777     0.743     0.753      2113



## Better than old model for churn 1

## Using stacking method for better prediction

In [45]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression

In [50]:
meta_model = LogisticRegression(max_iter=1000,class_weight='balanced', random_state=42)

In [51]:
estimators = [
    ('random_forest', rf_model),
    ('xgboost', xgb_model)
]

stack_model = StackingClassifier(
    estimators=estimators,
    final_estimator=meta_model,
    cv=5,                     # 5-fold stacking
    n_jobs=-1,
    passthrough=False         # use only base model predictions
)


In [52]:
stack_model.fit(X_train, y_train)

,estimators,"[('random_forest', ...), ('xgboost', ...)]"
,final_estimator,LogisticRegre...ndom_state=42)
,cv,5
,stack_method,'auto'
,n_jobs,-1
,passthrough,False
,verbose,0
,n_estimators,300
,criterion,'gini'
,max_depth,15
,min_samples_split,2


In [53]:
y_pred = stack_model.predict(X_test)
y_proba = stack_model.predict_proba(X_test)[:, 1]

print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("AUC:", roc_auc_score(y_test, y_proba))
print(classification_report(y_test, y_pred, digits=3))


Confusion Matrix:
 [[1225  314]
 [ 190  384]]
AUC: 0.8333797456604476
              precision    recall  f1-score   support

           0      0.866     0.796     0.829      1539
           1      0.550     0.669     0.604       574

    accuracy                          0.761      2113
   macro avg      0.708     0.732     0.717      2113
weighted avg      0.780     0.761     0.768      2113



In [54]:
import joblib
joblib.dump(stack_model, 'stacked_model.pkl')

['stacked_model.pkl']

### It is working bad then random forest and xgboost in predicting 1 which is more imp

In [56]:
from sklearn.model_selection import StratifiedKFold, cross_val_predict

In [58]:
# base models (already trained/tuned definitions)
rf = rf_model   # your RandomForestClassifier instance
xgb = xgb_model # your XGBClassifier instance

# 1) Create OOF probability features for training set
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# get OOF probs (shape: n_samples, ) for positive class
rf_oof = cross_val_predict(rf, X_train, y_train, cv=skf, method='predict_proba', n_jobs=-1)[:,1]
xgb_oof = cross_val_predict(xgb, X_train, y_train, cv=skf, method='predict_proba', n_jobs=-1)[:,1]

# stack features
X_meta_train = np.column_stack([rf_oof, xgb_oof])

# 2) Train base models on full training data so we can predict test
rf.fit(X_train, y_train)
xgb.fit(X_train, y_train)

# 3) Train meta model with class weighting
meta = LogisticRegression(max_iter=1000, class_weight='balanced', random_state=42)
meta.fit(X_meta_train, y_train)

# 4) Build meta features for test set using base models' predict_proba
rf_test_probs = rf.predict_proba(X_test)[:,1]
xgb_test_probs = xgb.predict_proba(X_test)[:,1]
X_meta_test = np.column_stack([rf_test_probs, xgb_test_probs])

# 5) Predict and evaluate
meta_probs = meta.predict_proba(X_meta_test)[:,1]
meta_pred = (meta_probs >= 0.5).astype(int)

print("Stacked Confusion Matrix (0/1):")
print(confusion_matrix(y_test, meta_pred))
print(classification_report(y_test, meta_pred, digits=3))
print("AUC:", roc_auc_score(y_test, meta_probs))

# 6) If recall too low, tune threshold
threshold = 0.4   # try 0.5, 0.45, 0.4,...
meta_pred_thresh = (meta_probs >= threshold).astype(int)
print("Thresh", threshold, "Confusion:", confusion_matrix(y_test, meta_pred_thresh))
print(classification_report(y_test, meta_pred_thresh, digits=3))

Stacked Confusion Matrix (0/1):
[[1231  308]
 [ 185  389]]
              precision    recall  f1-score   support

           0      0.869     0.800     0.833      1539
           1      0.558     0.678     0.612       574

    accuracy                          0.767      2113
   macro avg      0.714     0.739     0.723      2113
weighted avg      0.785     0.767     0.773      2113

AUC: 0.8337657603810793
Thresh 0.4 Confusion: [[1183  356]
 [ 152  422]]
              precision    recall  f1-score   support

           0      0.886     0.769     0.823      1539
           1      0.542     0.735     0.624       574

    accuracy                          0.760      2113
   macro avg      0.714     0.752     0.724      2113
weighted avg      0.793     0.760     0.769      2113



In [59]:
threshold = 0.5   # try 0.5, 0.45, 0.4,...
meta_pred_thresh = (meta_probs >= threshold).astype(int)
print("Thresh", threshold, "Confusion:", confusion_matrix(y_test, meta_pred_thresh))
print(classification_report(y_test, meta_pred_thresh, digits=3))

Thresh 0.5 Confusion: [[1231  308]
 [ 185  389]]
              precision    recall  f1-score   support

           0      0.869     0.800     0.833      1539
           1      0.558     0.678     0.612       574

    accuracy                          0.767      2113
   macro avg      0.714     0.739     0.723      2113
weighted avg      0.785     0.767     0.773      2113



In [61]:
threshold = 0.45  # try 0.5, 0.45, 0.4,...
meta_pred_thresh = (meta_probs >= threshold).astype(int)
print("Thresh", threshold, "Confusion:", confusion_matrix(y_test, meta_pred_thresh))
print(classification_report(y_test, meta_pred_thresh, digits=3))

Thresh 0.45 Confusion: [[1209  330]
 [ 172  402]]
              precision    recall  f1-score   support

           0      0.875     0.786     0.828      1539
           1      0.549     0.700     0.616       574

    accuracy                          0.762      2113
   macro avg      0.712     0.743     0.722      2113
weighted avg      0.787     0.762     0.770      2113



In [62]:
import joblib

# Save all parts separately
joblib.dump(rf, "random_forest_model.pkl")
joblib.dump(xgb, "xgboost_model.pkl")
joblib.dump(meta, "meta_logistic_model.pkl")

print("All models saved successfully.")


All models saved successfully.
